In [1]:
import torch
import numpy as np

In [2]:
torch.tensor([1,2,3])

tensor([1, 2, 3])

In [3]:
torch.tensor(np.array([1,2,3]))

tensor([1, 2, 3])

In [4]:
a=np.array([1,2,3])

t= torch.from_numpy(a)
c=t.numpy()
c

array([1, 2, 3])

In [6]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0) if torch.cuda.is_available() else None
torch.cuda.current_device()

0

In [9]:
device=torch.device('cuda')

a=torch.tensor([1,2,3],device=device)
a

tensor([1, 2, 3], device='cuda:0')

In [11]:
a.device

device(type='cuda', index=0)

In [14]:
import os
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import decode_image

In [21]:
from google.colab import files
uploaded = files.upload()  # upload mydata.zip


Saving annotations.csv to annotations.csv


In [16]:
class CustomImageDataset(Dataset):
  def __init__(self,annotations_file,img_dir,transform=None,target_transform=None):
    self.img_labels=pd.read_csv(annotations_file)
    self.img_dir=img_dir
    self.transform=transform
    self.target_transform=target_transform
  def __len__(self):
    return len(self.img_labels)
  def __getitem__(self, idx):
    img_path=os.path.join(self.img_dir,self.img_labels.iloc[idx,0])
    image=decode_image(img_path)
    label=self.img_labels.iloc[idx,1]
    if self.transform:
      image=self.transform(image)
    if self.target_transform:
      label=self.target_transform(label)
    return image,label

In [24]:
train_dataloader=CustomImageDataset("/content/annotations.csv","/content/drive/MyDrive/train")

In [25]:
train_dataloader.__len__()

1

## neural network in pytorch

#### load libraries

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [27]:
transform =transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
trainset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)

testset=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transform)
testloader=torch.utils.data.DataLoader(testset,batch_size=64,shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.83MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.2MB/s]


In [30]:
class SimpleDNN(nn.Module):
  def __init__(self):
    super(SimpleDNN,self).__init__()
    self.fc1=nn.Linear(28*28,256)
    self.fc2=nn.Linear(256,128)
    self.fc3=nn.Linear(128,10)
    self.relu=nn.ReLU()
    self.dropout=nn.Dropout(0.2)
  def forward(self,x):
    x=x.view(-1,28*28) #to flatten image
    x=self.relu(self.fc1(x))
    x=self.dropout(x)
    x=self.relu(self.fc2(x))
    x=self.dropout(x)
    x=self.fc3(x)
    return x

In [32]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=SimpleDNN().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
print(device)

cuda


In [37]:
epochs=5
for epoch in range(epochs):
  running_loss=0.0
  for images,labels in trainloader:
    images,labels=images.to(device),labels.to(device)
    optimizer.zero_grad()
    outputs=model(images)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss+=loss.item()
  print(f"epoch [{epoch+1}/{epochs}],Loss:{running_loss/len(trainloader):.4f}")

epoch [1/5],Loss:0.0879
epoch [2/5],Loss:0.0823
epoch [3/5],Loss:0.0815
epoch [4/5],Loss:0.0794
epoch [5/5],Loss:0.0780


In [38]:
correct=0
total=0
model.eval()
with torch.no_grad():
  for images,labels in testloader:
    images,labels=images.to(device),labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs,1)
    total+=labels.size(0)
    correct+=(predicted == labels).sum().item()
  print(f"test accuracy: {100*correct/total:.2f}%")

test accuracy: 97.95%
